### Importing Libs

In [ ]:
import numpy as np 
import pandas as pd 
import os
import gc
import time
from IPython.display import clear_output
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint as MC
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, LeakyReLU

### Viewing Data

In [ ]:
root = '/kaggle/input/rsna-str-pulmonary-embolism-detection'
for item in os.listdir(root):
    path = os.path.join(root, item)
    if os.path.isfile(path):
        print(path)

### Loading Data


In [ ]:
print('Reading train data...')
train = pd.read_csv("../input/rsna-str-pulmonary-embolism-detection/train.csv")
print(train.shape)
train.head()

In [ ]:
print('Reading test data...')
test = pd.read_csv("../input/rsna-str-pulmonary-embolism-detection/test.csv")
print(test.shape)
test.head()

In [ ]:
print('Reading sample data...')
ss = pd.read_csv("../input/rsna-str-pulmonary-embolism-detection/sample_submission.csv")
print(ss.shape)
ss.head()

## Checking Targets and Input Image

To make sure that we will be making the correct training targets, we check all ID from the sample submission.

In [ ]:
ids = ss.id
counter = [1 for _ in range(10)]
mapper = []
for i in ids:
    n = '_'.join(i.split('_')[1:])
    if n not in mapper:
        mapper.append(n)
    else:
        counter[mapper.index(n)] += 1
print("List of keys:")
print(mapper, sep='\n')
print()
print("Count of items per key:")
print(counter)

### The following fuction will return an image array from a DICOM image [Snipet](https://www.kaggle.com/eladwar/20-seconds-or-less).

In [ ]:
import vtk
from vtk.util import numpy_support
import cv2

reader = vtk.vtkDICOMImageReader()
def get_img(path):
    reader.SetFileName(path)
    reader.Update()
    _extent = reader.GetDataExtent()
    ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

    ConstPixelSpacing = reader.GetPixelSpacing()
    imageData = reader.GetOutput()
    pointData = imageData.GetPointData()
    arrayData = pointData.GetArray(0)
    ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
    ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')
    ArrayDicom = cv2.resize(ArrayDicom,(512,512))
    return ArrayDicom

Testing our Function

In [ ]:
#test read a dcom file and view it
fpath = "../input/rsna-str-pulmonary-embolism-detection/train/0003b3d648eb/d2b2960c2bbf/00ac73cfc372.dcm"
ds = get_img(fpath)

import matplotlib.pyplot as plt

#Convert dcom file to 8bit color
func = lambda x: int((2**15 + x)*(255/2**16))
int16_to_uint8 = np.vectorize(func)

def show_dicom_images(dcom):
    f, ax = plt.subplots(1,2, figsize=(16,20))
    data_row_img = int16_to_uint8(ds)
    ax[0].imshow(data_row_img, cmap=plt.cm.bone)
    ax[1].imshow(ds, cmap=plt.cm.bone)
    #print(data_row_img)
    ax[0].axis('off')
    ax[0].set_title('8-bit DICOM Image')
    ax[1].axis('off')
    ax[1].set_title('16-bit DICOM Image')
    plt.show()
    
show_dicom_images(ds)

# Defining Model Architecture 

Here I am using the first couple of layers from the famous pre-trained Xception model which will act like a feature-extractor and the rest of the layers will be custom trained to our dataset.

In [ ]:
inputs = Input((512, 512, 3))
#x = Conv2D(3, (1, 1), activation='relu')(inputs)
base_model = keras.applications.Xception(
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False

### The outer layers 

In [ ]:
outputs = base_model(inputs, training=False)
outputs = keras.layers.GlobalAveragePooling2D()(outputs)
outputs = Dropout(0.25)(outputs)
outputs = Dense(1024, kernel_initializer='uniform')(outputs)
outputs = LeakyReLU(alpha=0.3)(outputs)
outputs = Dense(256)(outputs)
outputs = LeakyReLU(alpha=0.1)(outputs)
outputs = Dense(64, activation='relu')(outputs)
ppoi = Dense(1, activation='sigmoid', name='pe_present_on_image')(outputs)
rlrg1 = Dense(1, activation='sigmoid', name='rv_lv_ratio_gte_1')(outputs)
rlrl1 = Dense(1, activation='sigmoid', name='rv_lv_ratio_lt_1')(outputs) 
lspe = Dense(1, activation='sigmoid', name='leftsided_pe')(outputs)
cpe = Dense(1, activation='sigmoid', name='chronic_pe')(outputs)
rspe = Dense(1, activation='sigmoid', name='rightsided_pe')(outputs)
aacpe = Dense(1, activation='sigmoid', name='acute_and_chronic_pe')(outputs)
cnpe = Dense(1, activation='sigmoid', name='central_pe')(outputs)
indt = Dense(1, activation='sigmoid', name='indeterminate')(outputs)

model = Model(inputs=inputs, outputs={'pe_present_on_image':ppoi,
                                      'rv_lv_ratio_gte_1':rlrg1,
                                      'rv_lv_ratio_lt_1':rlrl1,
                                      'leftsided_pe':lspe,
                                      'chronic_pe':cpe,
                                      'rightsided_pe':rspe,
                                      'acute_and_chronic_pe':aacpe,
                                      'central_pe':cnpe,
                                      'indeterminate':indt})

opt = keras.optimizers.Adam(lr=0.001)

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()
model.save('pe_detection_model.h5')
del model
K.clear_session()
gc.collect()

### Defining Callbacks

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath='../working/pe_detection_model.h5', monitor='val_loss', save_best_only=True, save_freq='epoch',verbose=1)
]

### Image Genrator

In [ ]:
def convert_to_rgb(array):
    array = array.reshape((512, 512, 1))
    return np.stack([array, array, array], axis=2).reshape((512, 512, 3))
    
def custom_dcom_image_generator(batch_size, dataset, test=False, debug=False):
    
    fnames = dataset[['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID']]
    
    if not test:
        Y = dataset[['pe_present_on_image', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1', 'leftsided_pe',
                     'chronic_pe', 'rightsided_pe', 'acute_and_chronic_pe', 'central_pe', 'indeterminate'
                    ]]
        prefix = 'input/rsna-str-pulmonary-embolism-detection/train'
        
    else:
        prefix = 'input/rsna-str-pulmonary-embolism-detection/test'
    
    X = []
    batch = 0
    for st, sr, so in fnames.values:
        if debug:
            print(f"Current file: ../{prefix}/{st}/{sr}/{so}.dcm")

        dicom = get_img(f"../{prefix}/{st}/{sr}/{so}.dcm")
        image = convert_to_rgb(dicom)
        X.append(image)
        
        del st, sr, so
        
        if len(X) == batch_size:
            if test:
                yield np.array(X)
                del X
            else:
                yield np.array(X), Y[batch*batch_size:(batch+1)*batch_size].values
                del X
                
            gc.collect()
            X = []
            batch += 1
        
    if test:
        yield np.array(X)
    else:
        yield np.array(X), Y[batch*batch_size:(batch+1)*batch_size].values
        del Y
    del X
    gc.collect()
    return

### Training proces

In [ ]:
history = {}
start = time.time()
debug = 0
batch_size = 1000
train_size = int(batch_size*0.9)

max_train_time = 3600 * 4 #hours to seconds of training


for n, (x, y) in enumerate(custom_dcom_image_generator(batch_size, train.sample(frac=1), False, debug)):
    
    if len(x) < 10: #Tries to filter out empty or short data
        break
        
    clear_output(wait=True)
    print("Training batch: %i - %i" %(batch_size*n, batch_size*(n+1)))
    
    model = load_model('../working/pe_detection_model.h5')
    hist = model.fit(
        x[:train_size], #Y values are in a dict as there's more than one target for training output
        {'pe_present_on_image':y[:train_size, 0],
         'rv_lv_ratio_gte_1':y[:train_size, 1],
         'rv_lv_ratio_lt_1':y[:train_size, 2],
         'leftsided_pe':y[:train_size, 3],
         'chronic_pe':y[:train_size, 4],
         'rightsided_pe':y[:train_size, 5],
         'acute_and_chronic_pe':y[:train_size, 6],
         'central_pe':y[:train_size, 7],
         'indeterminate':y[:train_size, 8]},

        callbacks = my_callbacks,

        validation_split=0.2,
        epochs=3,
        batch_size=8,
        verbose=debug
    )
    
    print("Metrics for batch validation:")
    model.evaluate(x[train_size:],
                   {'pe_present_on_image':y[train_size:, 0],
                    'rv_lv_ratio_gte_1':y[train_size:, 1],
                    'rv_lv_ratio_lt_1':y[train_size:, 2],
                    'leftsided_pe':y[train_size:, 3],
                    'chronic_pe':y[train_size:, 4],
                    'rightsided_pe':y[train_size:, 5],
                    'acute_and_chronic_pe':y[train_size:, 6],
                    'central_pe':y[train_size:, 7],
                    'indeterminate':y[train_size:, 8]
                   }
                  )
    
    try:
        for key in hist.history.keys():
            history[key] = np.concatenate([history[key], hist.history[key]], axis=0)
    except:
        for key in hist.history.keys():
            history[key] = hist.history[key]
            
    if time.time() - start >= max_train_time:
        print("Time's up!")
        break
        
    model.save('pe_detection_model.h5')
    del model, x, y, hist
    K.clear_session()
    gc.collect()

In [ ]:
for key in history.keys():
    if key.startswith('val'):
        continue
    else:
        epoch = range(len(history[key]))
        plt.plot(epoch, history[key]) #X=epoch, Y=value
        plt.plot(epoch, history['val_'+key])
        plt.title(key)
        if 'accuracy' in key:
            plt.axis([0, len(history[key]), -0.1, 1.1]) #Xmin, Xmax, Ymin, Ymax
        plt.legend(['train', 'validation'], loc='upper right')
        plt.show()

### Viewing Predictions


In [ ]:
predictions = {}
stopper = 3600 * 4 #4 hours limit for prediction
pred_start_time = time.time()

p, c = time.time(), time.time()
batch_size = 500
    
l = 0
n = test.shape[0]

for x in custom_dcom_image_generator(batch_size, test, True, False):
    clear_output(wait=True)
    model = load_model("../working/pe_detection_model.h5")
    preds = model.predict(x, batch_size=8, verbose=1)
    
    try:
        for key in preds.keys():
            predictions[key] += preds[key].flatten().tolist()
            
    except Exception as e:
        print(e)
        for key in preds.keys():
            predictions[key] = preds[key].flatten().tolist()
            
    l = (l+batch_size)%n
    print('Total predicted:', len(predictions['indeterminate']),'/', n)
    p, c = c, time.time()
    print("One batch time: %.2f seconds" %(c-p))
    print("ETA: %.2f" %((n-l)*(c-p)/batch_size))
    
    if c - pred_start_time >= stopper:
        print("Time's up!")
        break
    
    del model
    K.clear_session()
    
    del x, preds
    gc.collect()

In [ ]:
for key in predictions.keys():
    print(key, np.array(predictions[key]).shape)

In [ ]:
test_ids = []
for v in test.StudyInstanceUID:
    if v not in test_ids:
        test_ids.append(v)
        
test_preds = test.copy()
test_preds = pd.concat([test_preds, pd.DataFrame(predictions)], axis=1)
test_preds.to_csv('test_predictions.csv', index=False)
test_preds

In [ ]:
from scipy.special import softmax

label_agg = {key:[] for key in 
             ['id', 'negative_exam_for_pe', 'rv_lv_ratio_gte_1',
              'rv_lv_ratio_lt_1', 'leftsided_pe', 'chronic_pe',
              'rightsided_pe', 'acute_and_chronic_pe',
              'central_pe', 'indeterminate']
            }

for uid in test_ids:
    temp = test_preds.loc[test_preds.StudyInstanceUID ==uid]
    label_agg['id'].append(uid)
    
    n = temp.shape[0]
    #Check for any image level presence of PE of high confidence
    positive = any(temp.pe_present_on_image >= 0.5) #50% threshhold
    
    #Only one from positive, negative and indeterminate should have value>0.5
    #per exam
    if positive: 
        label_agg['indeterminate'].append(temp.indeterminate.min()/2)
        label_agg['negative_exam_for_pe'].append(0)
    else:
        if any(temp.indeterminate >= 0.5):
            label_agg['indeterminate'].append(temp.indeterminate.max())
            label_agg['negative_exam_for_pe'].append(1)
        else:
            label_agg['indeterminate'].append(temp.indeterminate.min()/2)
            label_agg['negative_exam_for_pe'].append(1)
    
    #I decided that the total ratio should be equal to 1, so I used softmax
    #We modify the weights by multiplying the bigger by 2 and dividing the smaller by 2
    a, b = temp[['rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1']].mean().values
    if a > b:
        a, b = a*2, b/2
    elif a < b:
        a, b = a/2, b*2
    a, b = softmax([a, b])
    if positive:
        label_agg['rv_lv_ratio_gte_1'].append(a)
        label_agg['rv_lv_ratio_lt_1'].append(b)
    else:
        label_agg['rv_lv_ratio_gte_1'].append(a/2)
        label_agg['rv_lv_ratio_lt_1'].append(b/2)
    
    #Next is for Chronic (C), Acute-Chronic (AC) and Acute (A) PE
    #We need to see if we got a high confidence value from either C or AC
    #If there is, we add it to a 50% based score for high confidence
    #and half weight for low confidence score
    if any(temp['acute_and_chronic_pe'] > 0.5): #50% confidence level
        label_agg['acute_and_chronic_pe'].append(0.5 + temp['acute_and_chronic_pe'].mean()/2)
        label_agg['chronic_pe'].append(temp['chronic_pe'].mean()/2)
        
    elif any(temp['chronic_pe'] > 0.5):
        label_agg['acute_and_chronic_pe'].append(temp['acute_and_chronic_pe'].mean()/2)
        label_agg['chronic_pe'].append(0.5 + temp['chronic_pe'].mean()/2)
        
    else: #Else, we set both to half values, as we declare the A as the value
        label_agg['acute_and_chronic_pe'].append(temp['acute_and_chronic_pe'].mean()/2)
        label_agg['chronic_pe'].append(temp['chronic_pe'].mean()/2)
    
    #for right, left, central, we use the same metric above
    for key in ['leftsided_pe', 'rightsided_pe', 'central_pe']:
        if positive:
            label_agg[key].append(0.5 + temp[key].mean()/2)
        else:
            label_agg[key].append(temp[key].mean()/2)

In [ ]:
uid = []
labels = []
df = pd.DataFrame(label_agg)
for key in ['negative_exam_for_pe', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1', 'leftsided_pe', 'chronic_pe',
            'rightsided_pe', 'acute_and_chronic_pe', 'central_pe', 'indeterminate']:
    for i in df.id:
        uid.append('_'.join([i, key]))
        labels.append(df.loc[df.id==i][key].values[0])
del df
gc.collect()

uid += test_preds.SOPInstanceUID.tolist()
labels += test_preds['pe_present_on_image'].tolist()

sub = pd.DataFrame({"id":uid, 'label':labels})
sub

Finally, we will save our submission into a file. But first, to make sure that we fill up any unpredicted variables (so that there will be no NaN in our values), we will fill up all missing values with 0.2 as a placeholder.

In [ ]:
sub.fillna(0.2, inplace=True)
sub.to_csv('submission.csv', index=False)

### Viewing one pred

In [ ]:
batch_size = 1
    
l = 0
n = test.shape[0]

for x in custom_dcom_image_generator(batch_size, test, True, False):
    clear_output(wait=True)
    model = load_model("../working/pe_detection_model.h5")
    preds = model.predict(x, batch_size=1, verbose=1)
    print(preds)
